In [2]:
import json
import pandas as pd
import numpy as np
import sys
import spacy
nlp = spacy.load("en_core_web_sm")
import nltk
from collections import Counter
import Levenshtein
import regex

tweets = pd.read_json("Data\gg2013.json")
awards = ["best screenplay - motion picture", "best director - motion picture",
          "best performance by an actress in a television series - comedy or musical",
          "best foreign language film", "best performance by an actor in a supporting role in a motion picture",
          "best performance by an actress in a supporting role in a series, mini-series or motion picture made for television",
          "best motion picture - comedy or musical",
          "best performance by an actress in a motion picture - comedy or musical",
          "best mini-series or motion picture made for television",
          "best original score - motion picture", "best performance by an actress in a television series - drama",
          "best performance by an actress in a motion picture - drama", "cecil b. demille award",
          "best performance by an actor in a motion picture - comedy or musical", "best motion picture - drama",
          "best performance by an actor in a supporting role in a series, mini-series or motion picture made for television",
          "best performance by an actress in a supporting role in a motion picture", "best television series - drama",
          "best performance by an actor in a mini-series or motion picture made for television",
          "best performance by an actress in a mini-series or motion picture made for television",
          "best animated feature film", "best original song - motion picture",
          "best performance by an actor in a motion picture - drama", "best television series - comedy or musical",
          "best performance by an actor in a television series - drama",
          "best performance by an actor in a television series - comedy or musical"]   
print("Done")

Done


In [173]:
def host():
    print("starting")
    hosts = {}
    count = 0
    for x, col in tweets.iterrows():
        tweet = col["text"]
        if "RT" in tweet: #Prevents getting the same tweet multiple times
            continue
        if ("host" in tweet.lower()):
            #To Speed Up (Will Need to Change)
#             count += 1
#             if (count%2 == 0 or count%3 == 0):
#                 continue
            doc = nlp(tweet)
            #Grabs proper pronouns (Removes certain key phrases too)
            pos = [tok.i for tok in doc if (tok.pos_ == "PROPN" and tok.text != 'RT' and "@" not in tok.text and "goldenglobe" not in tok.text.lower() and "globes" not in tok.text.lower() and "golden" not in tok.text.lower() and "globe" not in tok.text.lower() and "hosts" not in tok.text.lower() and "http" not in tok.text.lower())]
            consecutives = []
            current = []
            for elt in pos:
                if len(current) == 0:
                    current.append(elt)
                else:
                    if current[-1] == elt - 1:
                        current.append(elt)
                    else:
                        consecutives.append(current)
                        current = [elt]
                if (len(current) != 0):
                    consecutives.append(current)
                properPronouns = [doc[consecutive[0]:consecutive[-1]+1] for consecutive in consecutives]
                for h in (properPronouns):
                    if (len(h.text.split()) != 2): #Removes Pronouns that aren't 2 words
                        continue
                    if h.text.lower() in hosts:
                        hosts[h.text.lower()] += 1
                    else:
                        hosts[h.text.lower()] = 1
                        
    #Find the top 1-2 Names, those will be the people mentioned the most with the word host
    host_list = list(hosts.keys())

    p = np.percentile(list(hosts.values()),99)
    answer = []
    for key in hosts.keys():
        if (hosts[key] > p):
            answer.append(key)
    if (len(answer) >= 2):
        return answer[:2]
    else:
        return answer[0]
    

In [ ]:
print(host())

In [26]:
def award_names():
    print("STARTING")
    hosts = {}
    count = 0
    for x, col in tweets.iterrows():
        tweet = col["text"]
        if "RT" in tweet: #Prevents getting the same tweet multiple times
            continue
        if ("best " in tweet.lower()):# and(("nominated" in tweet.lower()) or ("won" in tweet.lower()))):                       
            doc = nlp(tweet)
            # print(doc)
#             for tok in doc:
#                 print(tok.text, tok.pos_, end=" ")
#             print()
#             continue
#             for (tok in doc):
#                 if (tok.text.lower() == "best"):
#                     prev = tok.i
#                     continue
#                 if (tok.i == prev + 1):
#                     if tok.po
            
            #Grabs proper pronouns (Removes certain key phrases too)
            # Took this out: tok.pos_ == "PROPN" and 
            pos = [tok.i for tok in doc if ((tok.pos_ == "PROPN" or tok.pos_ == "NOUN" or (tok.pos_ == "ADJ" and tok.text.lower() == "best")) and tok.text != 'RT' and "@" not in tok.text and "goldenglobe" not in tok.text.lower() and "globes" not in tok.text.lower() and "golden" not in tok.text.lower() and "globe" not in tok.text.lower() and "hosts" not in tok.text.lower() and "http" not in tok.text.lower())]
            consecutives = []
            current = []
            for elt in pos:
                if len(current) == 0:
                    current.append(elt)
                else:
                    if current[-1] == elt - 1:
                        current.append(elt)
                    else:
                        consecutives.append(current)
                        current = [elt]
                if (len(current) != 0):
                    consecutives.append(current)
                words = [doc[consecutive[0]:consecutive[-1]+1] for consecutive in consecutives]
                for h in (words):
                    if (h.text.lower()[0:4] != "best"):
                        continue
                    if h.text.lower() in hosts:
                        hosts[h.text.lower()] += 1
                    else:
                        hosts[h.text.lower()] = 1
                        
    #Find the top 1-2 Names, those will be the people mentioned the most with the word best
    host_list = list(hosts.keys())
    
    print(hosts)
    #p = np.percentile(list(hosts.values()),50)
    answer = []
    for key in hosts.keys():
        answer.append(key)
    return answer[1:28]

In [27]:
print(award_names())

STARTING
{'best': 12317, 'best people': 41, 'best use': 30, 'best dress': 189, 'best phrase': 17, 'best looks': 49, 'best choice': 23, 'best line': 378, 'best actress': 8115, 'best actress nominee': 3, 'best actress nominee jennifer': 4, 'best actress nominee jennifer lawrence': 23, 'best actor': 9400, 'best movie': 647, 'best movie drama': 103, 'best lauer': 11, 'best nights': 13, 'best actress drama': 197, 'best dresses': 40, 'best hair': 40, 'best screenplay': 1672, 'best song': 1215, 'best director': 3633, 'best look': 95, 'best look tonight': 7, 'best drama-': 11, 'best shows': 15, 'best friends': 209, 'best supporting': 681, 'best supporting actress': 3934, 'best actress tonight': 30, 'best hostesses': 15, 'best playoff': 2, 'best playoff weekend': 15, 'best dressed': 86, 'best food': 5, 'best duo': 33, 'best thing': 296, 'best ily': 2, 'best ily #': 3, 'best part': 1145, 'best #': 316, 'best james': 4, 'best james cameron': 6, 'best james cameron joke': 13, 'best things': 20, 'b

In [ ]:
def fuzzy_match(base, candidate, threshold):
    dist = Levenshtein.distance(base, candidate)
    base_len = len(base_str)
    return (dist <= round(base_len * threshold))

In [23]:
def award_winners():
    print("STARTING")
    winners = {}
    count = 0
    for x, col in tweets.iterrows():
        tweet = col["text"]
        if "RT" in tweet: #Prevents getting the same tweet multiple times
            continue
        for award in awards:
            if (award in tweet.lower() and "won" in tweet.lower()):
                #print(tweet)
                doc = nlp(tweet)
                pos = [tok.i for tok in doc if (tok.pos_ == "PROPN" and tok.text != 'RT' and "@" not in tok.text and "#" not in tok.text and "goldenglobe" not in tok.text.lower() and "globes" not in tok.text.lower() and "golden" not in tok.text.lower() and "globe" not in tok.text.lower() and "hosts" not in tok.text.lower() and "http" not in tok.text.lower())]
                consecutives = []
                current = []
                for elt in pos:
                    if len(current) == 0:
                        current.append(elt)
                    else:
                        if current[-1] == elt - 1:
                            current.append(elt)
                        else:
                            consecutives.append(current)
                            current = [elt]
                    if (len(current) != 0):
                        consecutives.append(current)
                    properPronouns = [doc[consecutive[0]:consecutive[-1]+1] for consecutive in consecutives]
                    for h in (properPronouns):
                        if ("best" in h.text.lower() or "hbo" in h.text.lower()):
                            continue
                        if (h.text.lower() in award):
                            continue
                        if (("act" in award or "director" in award or "cecil" in award) and (len(h.text.split()) != 2)):
                            continue
                        if award in winners:
                            if h.text.lower() in winners[award]:
                                winners[award][h.text.lower()] += 1
                            else:
                                winners[award][h.text.lower()] = 1
                        else:
                            winners[award] = {}
                            winners[award][h.text.lower()] = 1
    
    winn = {}
    for key in winners:
        winn[key] = list(winners[key])[0]
    print("done")
    print(winn)
award_winners()

STARTING
done
{'best original song - motion picture': 'skyfall', 'best animated feature film': 'brave', 'best motion picture - comedy or musical': 'les', 'best foreign language film': 'amour', 'best performance by an actor in a motion picture - drama': 'daniel day', 'best motion picture - drama': 'argo'}
